In [3]:
import html

def make_element(name, value, **attrs):
    keyvals = [' %s="%s"' % item for item in attrs.items()]
    attr_str = ''.join(keyvals)
    element = f'<{name}{attr_str}>{value}</{html.escape(value)}>'
    return element

# Пример
# Создает '<item size="large" quantity="6">Albatross</item>'
print(make_element('item', 'Albatross', size='large', quantity=6))

# Создает '<p>&lt;spam&gt;</p>'
print(make_element('p', '<spam>'))

<item size="large" quantity="6">Albatross</Albatross>
<p><spam></&lt;spam&gt;>


In [5]:
funcs = [lambda x: x+n for n in range(5)]
for f in funcs:
    print(f(0))

4
4
4
4
4


In [6]:
funcs = [lambda x, n=n: x+n for n in range(5)]
for f in funcs:
    print(f(0))

0
1
2
3
4


In [12]:
from functools import partial

def spam(a, b, c, d):
    print(a, b, c, d)

s1 = partial(spam, 1)
print(s1(2,3,4)) 

s2 = partial(spam, d=42) 
print(s2(1,2,3))

s3 = partial(spam, 1, 2, d=42)
print(s3(3))

1 2 3 4
None
1 2 3 42
None
1 2 3 42
None


In [11]:
import math 

points = [(1, 2), (3, 4), (5, 6), (7, 8)]

def distance(p1, p2):
    x1, y1 = p1
    x2, y2 = p2
    return math.hypot(x2 - x1, y2 - y1)

pt = (4, 3)
points.sort(key=partial(distance, pt))
print(points)

[(3, 4), (1, 2), (5, 6), (7, 8)]


In [4]:
def apply_async(func, args, *, callback):
    # Вычислить результат
    result = func(*args)
    # Вызвать функцию обратного вызова с результатом
    callback(result)
    
def print_result(result):
    print('Got:', result)
    
def add(x, y):
    return x + y

apply_async(add, (2, 3), callback=print_result)
apply_async(add, ('hello', 'world'), callback=print_result)

Got: 5
Got: helloworld


In [6]:
class ResultHandler:
    def __init__(self):
        self.sequence = 0
    
    def handler(self, result):
        self.sequence += 1
        print(f'[{self.sequence}] Got: {result}')
        
r = ResultHandler()

apply_async(add, (2, 3), callback=r.handler)
apply_async(add, ('hello', 'world'), callback=r.handler)

[1] Got: 5
[2] Got: helloworld


In [7]:
def make_handler():
    sequence = 0
    def handler(result):
        nonlocal sequence
        sequence += 1
        print(f'[{sequence}] Got: {result}')
    return handler

handler = make_handler()

apply_async(add, (2, 3), callback=handler)
apply_async(add, ('hello', 'world'), callback=handler)

[1] Got: 5
[2] Got: helloworld


In [10]:
def make_handler():
    sequence = 0
    while True:
        result = yield
        sequence += 1
        print(f'[{sequence}] Got: {result}')
        
handler = make_handler()

next(handler) # Продвигаемся к yield

apply_async(add, (2, 3), callback=handler.send)
apply_async(add, ('hello', 'world'), callback=handler.send)

[1] Got: 5
[2] Got: helloworld


In [14]:
class SequenceNo:
    def __init__(self):
        self.sequence = 0
        
def handler(result, seq):
    seq.sequence += 1
    print(f'[{seq.sequence}] Got: {result}')
    
seq = SequenceNo()

apply_async(add, (2, 3), callback=partial(handler, seq=seq))
apply_async(add, ('hello', 'world'), callback=partial(handler, seq=seq))

apply_async(add, (2, 3), callback=lambda r: handler(r, seq))

[1] Got: 5
[2] Got: helloworld
[3] Got: 5


In [19]:
from queue import Queue
from functools import wraps

def inlined_async(func):
    @wraps(func)
    def wrapper(*args):
        f = func(*args)
        result_queue = Queue()
        result_queue.put(None)
            
        while True:
            result = result_queue.get()
            try:
                a = f.send(result)
                apply_async(a.func, a.args, callback=result_queue.put)
            except StopIteration:
                break
    return wrapper


def apply_async(func, args, *, callback):
    # Вычисляем результат
    result = func(*args)
    # Вызываем функцию обратного вызова с результатом
    callback(result)
    

def add(x, y):
    return x + y


class Async:
    def __init__(self, func, args):
        self.func = func
        self.args = args


@inlined_async
def test():
    r = yield Async(add, (2, 3))
    print(r)
    r = yield Async(add, ('hello', 'world'))
    print(r)
    for n in range(10):
        r = yield Async(add, (n, n))
        print(r)
    print('Goodbye')
    

test()

5
helloworld
0
2
4
6
8
10
12
14
16
18
Goodbye


In [25]:
def sample():
    n = 0
    # Функция-замыкание
    def func():
        print('n =', n)
        
    # Методы доступа к n
    def get_n():
        return n
    
    def set_n(value):
        nonlocal n
        n = value
        
    # Прикрепление в качестве атрибутов функции
    func.get_n = get_n
    func.set_n = set_n
    return func

f = sample()
f()
f.set_n(10)
f()
f.get_n()

n = 0
n = 10


10

In [28]:
import sys

class ClosureInstance:
    def __init__(self, locals=None):
        if locals is None:
            locals = sys._getframe(1).f_locals
        # Обновить словарь экземпляра вызываемыми объектами
        self.__dict__.update((key, value) for key, value in locals.items() if callable(value))
        
    # перегружаем специальные методы
    def __len__(self):
        return self.__dict__['__len__']()
    

# Пример использования
def Stack():
    items = []
    
    def push(item):
        items.append(item)
    
    def pop():
        return items.pop()
    
    def __len__():
        return len(items)
    
    return ClosureInstance()


s = Stack()
print(s)
s.push(10)
s.push(20)
s.push('Hello')
print(len(s))
print(s.pop())
print(s.pop())
print(s.pop())
            
    

3
Hello
20
10


In [1]:
def test():
    a = 13
    loc = locals()
    exec('b = a + 1')
    b = loc['b']
    print(b)
    
test()

14


In [13]:
def countdown(n):
    while n > 0:
        print('T-minus', n)
        n -= 1
    print('Blastoff!')
    

import dis
dis.dis(countdown)

  2           0 LOAD_FAST                0 (n)
              2 LOAD_CONST               1 (0)
              4 COMPARE_OP               4 (>)
              6 POP_JUMP_IF_FALSE       17 (to 34)

  3     >>    8 LOAD_GLOBAL              0 (print)
             10 LOAD_CONST               2 ('T-minus')
             12 LOAD_FAST                0 (n)
             14 CALL_FUNCTION            2
             16 POP_TOP

  4          18 LOAD_FAST                0 (n)
             20 LOAD_CONST               3 (1)
             22 INPLACE_SUBTRACT
             24 STORE_FAST               0 (n)

  2          26 LOAD_FAST                0 (n)
             28 LOAD_CONST               1 (0)
             30 COMPARE_OP               4 (>)
             32 POP_JUMP_IF_TRUE         4 (to 8)

  5     >>   34 LOAD_GLOBAL              0 (print)
             36 LOAD_CONST               4 ('Blastoff!')
             38 CALL_FUNCTION            1
             40 POP_TOP
             42 LOAD_CONST               0 (

In [14]:
countdown.__code__.co_code

b'|\x00d\x01k\x04r\x11t\x00d\x02|\x00\x83\x02\x01\x00|\x00d\x038\x00}\x00|\x00d\x01k\x04s\x04t\x00d\x04\x83\x01\x01\x00d\x00S\x00'

In [18]:
import opcode

c = countdown.__code__.co_code

print(opcode.opname[c[0]])
print(opcode.opname[c[3]])

LOAD_FAST
POP_TOP


In [33]:
import importlib

math = importlib.import_module('math')
print(math.sin(2))

mod = importlib.import_module('urllib.request')
u = mod.urlopen('http://www.python.org')
print(u.status)

0.9092974268256817
200


In [ ]:
import imp
import urllib.request
import sys

def load_module(url):
    u = urllib.request.urlopen(url)
    source = u.read().decode('utf-8')
    mod = sys.modules.setdefault(url, imp.new_module(url))
    code = compile(source, url, 'exec')
    mod.__file__ = url
    mod.__package__ = ''
    exec(code, mod.__dict__)
    return mod

print(load_module('http://localhost:15000/neuron.py'))

In [4]:
import imp
import sys

m = imp.new_module('spam')
print(m)
print(m.__name__)

m2 = sys.modules.setdefault('spam', imp.new_module('spam'))
print(m2)

<module 'spam'>
spam
<module 'spam'>


In [6]:
import math 

m = sys.modules.setdefault('math', imp.new_module('math'))
print(m)
print(m.sin(2))
print(m.cos(2))

<module 'math' (built-in)>
0.9092974268256817
-0.4161468365471424


In [7]:
from pprint import pprint

pprint(sys.meta_path)

 <class '_frozen_importlib.BuiltinImporter'>,
 <class '_frozen_importlib.FrozenImporter'>,
 <class '_frozen_importlib_external.PathFinder'>,
